# Algoritmo One-Sided Jacobi para SVD



La descomposición en valores singulares de una matriz (SVD por sus siglas en inglés) consiste hallar matrices que permitan su factorización; en concreto, se refiere a hallar dos matrices ortonormales U y V asociadas al dominio y co-dominio de $A$, junto con una matriz diagonal $\Sigma$ cuyas entradas se encuentra determinadas por los **valores singulares** de la matriz en cuestión. 


**Objetivos**

Realizar la implementación en Python de:

1) El algoritmo One-Sided Jacobi para aproximar la descomposición SVD de una matriz,

2) Utilizar el resultado anterior para realizar PCA

Cabe mencionar que la implementación del algoritmo One-Sided Jacobi presentada a continuación fue basada en el desarrollo (en R) realizado por nuestros compañeros de clase, que puede ser consultada en https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/jupyter/Ex_CM_SVD.ipynb


## 1. Aspectos teóricos

### 1.1 Descomposición SVD

Por resultados de álgebra lineal y análisis matemático, se sabe que para una matriz ![$A \in \mathbb{R}^{mxn}$](https://render.githubusercontent.com/render/math?math=A%20%5Cin%20%5Cmathbb%7BR%7D%5E%7Bmxn%7D&mode=inline) es posible encontrar una factorización de ésta en términos de matrices ![$U \in \mathbb{R}^{mxm}, V \in \mathbb{R}^{nxn}$](https://render.githubusercontent.com/render/math?math=U%20%5Cin%20%5Cmathbb%7BR%7D%5E%7Bmxm%7D%2C%20V%20%5Cin%20%5Cmathbb%7BR%7D%5E%7Bnxn%7D&mode=inline) ortogonales tales que: ![$A = U\Sigma V^T$](https://render.githubusercontent.com/render/math?math=A%20%3D%20U%5CSigma%20V%5ET&mode=inline) con ![$\Sigma = diag(\sigma_1, \sigma_2, \dots, \sigma_p) \in \mathbb{R}^{mxn}$](https://render.githubusercontent.com/render/math?math=%5CSigma%20%3D%20diag%28%5Csigma_1%2C%20%5Csigma_2%2C%20%5Cdots%2C%20%5Csigma_p%29%20%5Cin%20%5Cmathbb%7BR%7D%5E%7Bmxn%7D&mode=inline), ![$p = \min\{m,n\}$](https://render.githubusercontent.com/render/math?math=p%20%3D%20%5Cmin%5C%7Bm%2Cn%5C%7D&mode=inline) y ![$\sigma_1 \geq \sigma_2 \geq \dots \geq \sigma_p \geq 0$](https://render.githubusercontent.com/render/math?math=%5Csigma_1%20%5Cgeq%20%5Csigma_2%20%5Cgeq%20%5Cdots%20%5Cgeq%20%5Csigma_p%20%5Cgeq%200&mode=inline).

Típicamente $U$, $\Sigma$ y $V$ se suelen asociar a acciones de $A$ sobre la bola unitaria y la imagen de ésta sobre $A$.

Es importante que existen representaciones alternas dicha factorización, pero similarmente tales se basan en encontrar matrices ortogonales asociadas al dominio y condominio de $A$, junto con sus valores singulares. Véase https://en.wikipedia.org/wiki/Singular_value_decomposition#Reduced_SVDs

### 1.2.1 Algoritmo One-Sided Jacobi para aproximar la descomposición SVD

**Nota:** Los detalles del algoritmo a continuación se han tomado de la nota https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/III.computo_matricial/3.3.d.SVD.ipynb

Para obtener versiones numéricas de la descomposición SVD de una matriz existe un método iterativo conocido como el **algoritmo One-Sided Jacobi** que se basa en la aplicación sucesiva de rotaciones de Givens, para ortogonalizar las columnas de las matrices involucradas, así como estimar los respectivos valores singulares. En concreto, la idea en que se basa tal método es multiplicar a la matriz ![$A \in \mathbb{R}^{m \times n}$](https://render.githubusercontent.com/render/math?math=A%20%5Cin%20%5Cmathbb%7BR%7D%5E%7Bm%20%5Ctimes%20n%7D&mode=inline) por la derecha de forma repetida por matrices ortogonales de nombre **rotaciones Givens** hasta que se converja a ![$U \Sigma$](https://render.githubusercontent.com/render/math?math=U%20%5CSigma&mode=inline).


Es decir, se trata de el algoritmo que realiza interacciones sobre las matrices $A,V$ de la forma: $A^{(k+1)} = A^{(k)}U^{(k)}, V^{(k+1)} = V^{(k)}U^{(k)}$ para $k>0$ donde: $U^{(k)}$ son matrices de rotación del plano $(i,j)$. Esto es, una identidad matriz identidad compatible con las dimensiones de $A$ y $V$ pero que actúa únicamente sobre las entradas dadas los índices $i$ y $j$ a través de las fórmulas que determinan una rotación de Givens

$$u_{ii}^{(k)} = cos(\theta), \quad u_{ij}^{(k)} = sen(\theta)$$

$$u_{ji}^{(k)}=-sen(\theta), \quad u_{jj}^{(k)}=cos(\theta).$$


Es decir, la multiplicación $A^{(k)}U^{(k)}$ sólo involucra a $2$ columnas de $A^{(k)}$:

$$(a_i^{(k+1)} a_j^{(k+1)}) = (a_i^{(k)} a_j^{(k)})\left[ \begin{array}{cc}
cos(\theta) & sen(\theta)\\
-sen(\theta) & cos(\theta)
\end{array}
\right]$$


Al realizarse rotaciones con referencias a las entradas $(i,j)$ de $A^{(k)}$, en el proceso iterativo debe existir una secuencia, denominada *sweep*. Cada *sweep* consiste de como máximo $\frac{n(n-1)}{2}$ rotaciones a aplicarse y es dependiente de cuántas columnas son o no ortogonales dentro del proceso (iterativo), recordando que en cada *sweep* se ortogonalizan $2$ columnas empleando una rotación de Givens.

Al finalizar el algoritmo los valores singulares calculados son las normas Euclidianas de cada columna de $A^{(k)}$. Las columnas normalizadas de $A^{(k)}$ aproximarán a las columnas de $U$. 

**Notas:**


* El ángulo $\theta$ de la rotación de Givens en cada sweep se debe eligir de acuerdo a $2$ criterios:

    1) El ángulo es $0$ si $a_i^{T(k)}a_j^{(k)}=0$ y por lo tanto las columnas $i,j$ son ortogonales y no se hace rotación.
    
    2) $\theta \in (\frac{-\pi}{4}, \frac{pi}{4})$ tal que $a_i^{T(k+1)}a_j^{(k+1)}=0$ y para este caso se calculan $\xi, t, cs,sn$ (las variables $cs, sn$ hacen referencia a $cos(\theta), sen(\theta)$.
* La convergencia del algoritmo depende del ángulo de rotación, pero se saber que si los *sweep*  se realizan secuencialmente siguiente el ordenamiento cíclico por renglones considerando pares de columnas $(1,2), (1,3), \dots, (1,n), (2,3), (2,4), \dots, (n-1,n)$, tal ordenamiento si $|\theta| \leq \frac{\pi}{4}$.

**Consideraciones sobre la implementación:**
* El número de *sweeps* máximos a realizarse se debe controlarse en la implementación,
* El número de columnas ortogonales también es un parámetro a controlar en el algoritmo,
* La implementación requerirá entonces el desarrollo de funciones capaces de:
 * Determinar si dos vectores (columnas) son ortogonales dada cierta tolerancia,
 * Un mecanismo para generar la secuencia de índices $(1,2), (1,3), \dots, (1,n), (2,3), (2,4), \dots, (n-1,n)$ en la que deben correr los *sweeps*,
 * Calcular el valor de la función signo sobre números reales, puesto que se involucra en la expresión de las rotaciones de Givens,


De todo lo anterior, el Algoritmo One-Sided Jacobi se puede describir como sigue:

**Algoritmo One-Sided Jacobi**

Entrada: 

* matriz $A \in \mathbb{R}^{m \times n}$: matriz a la que se le calculará su SVD.

* $TOL$ controla la convergencia del método.

* $maxsweeps$ número máximo de sweeps (descritos en los comentarios de abajo).

Salida: 

* matrices $V \in \mathbb{R}^{n \times n}$ ortogonal, $W \in \mathbb{R}^{m \times n}$ representada en el algoritmo por $A^{(k)}$ para un valor de $k$ controlado por la convergencia (descrita en los comentarios de abajo).


Nota: se utilizará la notación $A^{(k)}=[a_1^{(k)} a_2^{(k)} \cdots a_n^{(k)}]$ con cada $a_i^{(k)}$ como $i$-ésima columna de $A$ y $k$ representa el *sweep*.

Definir $A^{(0)}=A$, $V^{(0)}=I_n$ (*sweep* $=0$). 

* Mientras no se haya llegado al número máximo de sweeps ($k \leq maxsweeps$) o el número de columnas ortogonales sea menor a $\frac{n(n-1)}{2}$:

    * Para todos los pares con índices $i<j$ generados con alguna metodología (descrita en la sección de abajo) y para k desde 0 hasta convergencia:
    
        * Revisar si las columnas $a_i^{(k)}, a_j^{(k)}$ de $A^{(k)}$ son ortogonales (el chequeo se describe en los comentarios). Si son ortogonales se incrementa por uno la variable $num\text{_}columnas\text{_}ortogonales$. Si no son ortogonales:
        
            * Calcular $\left[ \begin{array}{cc}
a & c\\
c & b
\end{array}
\right]$ la submatriz $(i,j)$ de $A^{T(0)}A^{(0)}$ donde: $a = ||a_i^{(k)}||_2^2, b=||a_j^{(k)}||_2^2, c=a_i^{T(k)}a_j^{(k)}$. 

            * Calcular la rotación Givens que diagonaliza $\left[ \begin{array}{cc}
a & c\\
c & b
\end{array}
\right]$ definiendo: $\xi = \frac{b-a}{2c}, t=\frac{signo(\xi)}{|\xi| + \sqrt{1+\xi^2}}, cs = \frac{1}{\sqrt{1+t^2}}, sn = cs*t$. Recordar que $signo(a) = \begin{cases}
1 &\text{ si } a \geq 0 ,\\
-1 &\text{ si } a < 0
\end{cases}$.
    
            * Actualizar las columnas $i,j$ de $A^{(k)}$. Para $\ell$ de $1$ a $n$:
    
                * $temp = A^{(k)}_{\ell i}$
        
                * $A_{\ell i}^{(k)} = cs*temp - sn*A_{\ell j}^{(k)}$
        
                * $A_{\ell j}^{(k)} = sn*temp + cs*A_{\ell j}^{(k)}$
        
            * Actualizar a la matriz $V^{(k)}$. Para $\ell$ de $1$ a $n$:
    
                * $temp = V_{\ell i}^{(k)}$
        
                * $V_{\ell i}^{(k)} = cs*temp - sn*V_{\ell j}^{(k)}$
        
                * $V_{\ell j}^{(k)} = sn*temp + cs*V_{\ell j}^{(k)}$
                
    * Incrementar por uno la variable $k$ que cuenta el número de sweeps.


## 2. Funciones auxiliares

### 2.1 Verificación de ortogonalidad entre vectores

**Objetivo:**

Dados dos vectores $u,v\in \mathbb{R}^n$ y un parámetro $TOL$ de tolerancia, generar una función *ortogonal* que verifique si el producto punto de la versión normalizada de ambos vectores se considera ortogonal; es decir, que evalue la verdad o falsedad de la afirmación:

$$ \frac{u \cdot v^T}{|| u ||_2 || v ||_2 } < TOL$$

**Notas:** 

* $TOL$ debe ser un parámetro (número real positivo) de esta función para que el usuario pueda definir a voluntad,

* La norma de los vectores a involucrarse en la definición es la norma-2,
* La función debe regresar un valor booleano, que nos diga si dada la tolerancia la versión normalizada de ambos vectores se considera ortogonal.


In [1]:
def verificar_ortogonalidad(u,v,tol = 10^-8):
    if np.linalg.norm(u) <= tol or np.linalg.norm(v) <= tol:
        resultado = 1
    elif ((np.dot(u,v))/(np.linalg.norm(u)*np.linalg.norm(v))) <= tol:
        resultado = 1
    else:
        resultado = 0
    return resultado

In [28]:
def ortogonal(u,v,TOL=10^-8):
  # Verifica si dos vectores son ortogonales, de acuerdo a cierto nivel de tolerancia, 
  # arrojando un 1 si lo es, y un 0 si no lo es.
  # Args: 
  #   u (vector): vector de dimension n,
  #   v (vector): vector de dimension n, 
  #   TOL (numeric): real positivo, que sirve como parametro de tolerancia para evaluar ortogonalidad de u y v. 
  #   Notas: 
  #   1) Valor por default TOL es 10^-8
  #   2) Se sugiere una TOL mayor a 10^-32.
  # Returns: 
  #   Valor booleano 0 (no son ortogonales), 1 (son ortogonales)
    if (np.linalg.norm(u,2) < TOL or np.linalg.norm(v,2) < TOL):
        ort =0 
    else: 
        if(np.dot(u,v)/(np.linalg.norm(u,2)*np.linalg.norm(v,2)) < TOL):
            ort=1
        else:
            ort=0  
    
    return ort


### 2.2 Función signo

**Objetivo:**

Generar una función *signo* que verifique el signo de un número real; es decir:

$$signo(a) = \begin{cases} 1 & \mbox{ si } a \geq 0 \\ -1 & \mbox{ si } a < 0 \end{cases}$$

Observemos que `numpy` ya cuenta con la función `numpy.sign()`, sin embargo, `numpy.sign(0)` devuelve cero, por lo que proponemos una versión ligeramente modificada:


In [3]:
def signo(x):
    if x<0:
        sig = -1
    else:
        sig = 1
    return sig

In [4]:
signo(0)

1

## 3. Algoritmo SVD


### 3.1 One-sided Jacobi numerical aproximación

**Propósito:**

Dada una matriz $A \in \mathbb{R}^{ m \times n}$, crear una función *SVD_jacobi_aprox* que permita obtener aproximaciones numéricas asociadas a la descomposición SVD de esta,. En concreto, si la descomposición de $A$ está dada por las consabidas matrices $U \in \mathbb{R}^{m \times m}$, $\Sigma\in \mathbb{R}^{n \times n}$ y $V \in \mathbb{R}^{n \times n}$,  queremos obtener versiones numéricas de 1) $V$ y 2) $W:=U \Sigma$.

Ello se debe realizar con base en las siguientes especificaciones, que circunscriben el Algoritmo One-sided Jacobi para la descomposición SVD:

**Entradas:**

* Matriz $A \in \mathbb{R}^{ m \times n}$,
* TOL (número real, parámetro de tolerancia prefijado en $10^{-8}$),
* *maxsweeps* (entero mayor a cero). **Nota:** Las rotaciones que se aplicarán sucesivamente para ortogonalizar las matrices a partir de A se realizan en una secuencia con nombre *sweep*. Cada *sweep* consiste de como máximo $\frac{n(n+1)}{2}$ rotaciones (pues depende de cuántas columnas son o no ortogonales) y en cada *sweep* se ortogonalizan 2 columnas. El número de *sweeps* a realizar se controla con esta variable.

**Salidas:**

* $V \in \mathbb{R}^{n \times n}$ (dadas por $V\leftarrow V^k$, al final del proceso iterativo),
* $S \in \mathbb{R}^{n \times n}$ (dada por norma de $A^k$, al final del proceso iterativo),
* $U \in \mathbb{R}^{m \times n}$ (dada por $U\leftarrow A^k$ a la que se le han normalizado las columnas, al final del proceso iterativo),

**Funciones del proyecto  que depende:**

Signo y ortogonal (véanse secciones 2.2 y 2.3)

**Reporte de revisión:**

Los reportes de revisión correspondiente a esta función se encuentra disponible para su consulta en el [vínculo 1](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/test/Rev_One-sided_Jacobi.ipynb) y [vínculo 2](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/test/Rev_One-sided_Jacobi_2.2.ipynb).

In [29]:
import numpy as np
def svd_jacobi_aprox(A,TOL,maxsweep):
    # Calcula la descomposición de una matriz A en sus componentes U, S V, 
    # utilizando el método de Jacobi para calcular la factorización SVD.De esta forma 
    # la matriz A queda descompuesta de la siguiente forma: A = U*S*t(V).
    # Args: 
    #    A (matriz): Matriz de entrada (nxm) de números reales a la que se le calculará la descomposición SVD.
    #    TOL (numeric): controla la convergencia del método, siendo un valor real de 10^-8 (sugerido en la nota 3.3.d.SVD)
    #    Nota: Se sugiere una TOL mayor a 10^-32.
    #    maxsweep (numeric): número máximo de sweeps,donde cada sweep consiste de un número máximo(nmax)
    #    de rotaciones; y en cada sweep se ortogonalizan 2 columnas.
    # Returns: 
    #   Lista con 3 elementos, donde el primer elemento representa a la matriz U(nxm),el segundo a la matriz S(mxm) matriz diagonal
    #   y el tercero y último a la matriz V (mxm).En conjunto estas tres matrices componen la factorización SVD de la matriz de entrada A.
    # Nota: Esta función estima la SVD thin,la cual calcula unicamente las m columnas de U correspondientes a los m renglones de V. De esta
    # manera las columnas restantes de U no son calculadas, provocando una mejora significativa en velocidad de ejecución comparada con la 
    # la Full SVD. Referencia: https://en.wikipedia.org/wiki/Singular_value_decomposition#Thin_SVD.
    
    #dimensiones de A
    n = A.shape[1] #numero de columnas
    m = A.shape[0] #numero de filas
    nmax =n*(n-1)/2
    
    #inicializa valores del ciclo
    ak = A
    vk = np.identity(n, dtype = float) 
    sig = ''
    uk = ak
    num_col_ortogonal = 0
    k = 0
    stop = False
        
    
    while(k<=maxsweep & num_col_ortogonal<nmax):
        num_col_ortogonal =0
        for i in range(n-1):
            for j in range(i+1,n):
                col_j = ak[:,j]
                col_i = ak[:,i]

                #comprueba ortogonalidad  
                if(ortogonal(col_i,col_j,TOL)==1):
                    num_col_ortogonal =num_col_ortogonal+1
                else:
                    #calcula coeficientes de la matriz
                    a = np.dot(col_i,col_i)
                    b = np.dot(col_j,col_j)
                    c = np.dot(col_i,col_j)

                    #si c es cercano a cero no actualiza
                    if(c<TOL):
                        stop =True
                        break

                    #calcula la rotacion givens que diagonaliza
                    epsilon = (b-a)/(2*c)
                    t = signo(epsilon)/(abs(epsilon)+np.sqrt(1+epsilon**2))
                    cs = 1/np.sqrt(1+t**2)
                    sn = cs*t

                    #actualiza las columnas de la matriz ak
                    temp = ak[:,i] 
                    ak[:,i] = cs*temp-sn*ak[:,j]
                    ak[:,j] = sn*temp+cs*ak[:,j]


                    #actualiza las columnas de la matriz vk
                    temp = vk[:,i] 
                    vk[:,i] = cs*temp-sn*vk[:,j]
                    vk[:,j] = sn*temp+cs*vk[:,j]             
                #cierra else
            #cierra for j
            if(stop):
                stop = False
                break
            
        #cierra for i
        k = k+1
     #cierra while
    
      
    #Obtener sigma (normas euclidianas de columnas de ak)
    sig = np.linalg.norm(ak, axis=0)

    #Obtener U (columnas normalizadas de ak)
    for i in range(n):
        if (sig[i]<TOL):
            uk[:,i] = 0  
        else:
            uk[:,i] = ak[:,i]/sig[i]
        

    # Indices de sigma ordenada en forma decreciente para ordenar V,S,U
    #index <- order(sig,decreasing = TRUE)
    #vk <- vk[,index]
    #S <- np.diag(sig[index])
    #uk <- uk[,index]
    
    U = uk
    s = sig
    V= vk

    return U, s, V  

## Pruebas

In [19]:
np.random.seed(2020)
a = np.random.randn(9, 6)
b = np.random.randn(2, 7, 8, 3)

In [20]:
a

array([[-1.76884571,  0.07555227, -1.1306297 , -0.65143017, -0.89311563,
        -1.27410098],
       [-0.06115443,  0.06451384,  0.41011295, -0.57288249, -0.80133362,
         1.31203519],
       [ 1.27469887, -1.2143576 ,  0.31371941, -1.44482142, -0.3689613 ,
        -0.76922658],
       [ 0.3926161 ,  0.05729383,  2.08997884,  0.04197131, -0.04834072,
        -0.51315392],
       [-0.08458928, -1.21545008, -1.41293073, -1.48691055,  0.38222486,
         0.937673  ],
       [ 1.77267804,  0.87882801,  0.33171912, -0.30603567,  1.24026615,
        -0.21562684],
       [ 0.15592948,  0.09805553,  0.83209585,  2.04520542, -0.31681392,
        -1.31283291],
       [-1.75445746,  0.10209408, -1.36150208,  0.48178488, -0.20832874,
        -0.09186351],
       [ 0.70268816,  0.10365506,  0.62123638,  0.95411497,  2.03781352,
        -0.48445122]])

### Prueba con np.linalg.svd

In [21]:
u, s, vh = np.linalg.svd(a, full_matrices=False)

In [22]:
u.shape, s.shape, vh.shape

((9, 6), (6,), (6, 6))

In [23]:
n = a.shape[1] #numero de columnas
m = a.shape[0] #numero de filas
nmax =n*(n-1)/2
n

6

In [24]:
np.allclose(a, np.dot(u * s, vh)) #Returns True if two arrays are element-wise equal within a tolerance.

True

In [25]:
smat = np.diag(s)
np.allclose(a, np.dot(u, np.dot(smat, vh)))

True

In [26]:
np.allclose(a, u @ (s[..., None] * vh))

True

### Prueba con el método propuesto

In [30]:
U, S, V = svd_jacobi_aprox(a,1e-8,50)

In [31]:
U.shape, S.shape, V.shape

((9, 6), (6,), (6, 6))

In [32]:
Smat = np.diag(s)
np.allclose(a, np.dot(U, np.dot(Smat, V.T)))

False

In [33]:
np.allclose(u,U)

False

In [34]:
np.allclose(s,S)

False

In [35]:
np.allclose(vh,V)

False

In [36]:
s

array([4.61438141, 3.74993553, 2.69498261, 2.31903031, 1.44108343,
       1.19491148])

In [37]:
S

array([4.5496564 , 1.40066774, 1.2062377 , 2.59631441, 1.85522064,
       2.46400838])

In [38]:
A = a
#dimensiones de A
n = A.shape[1] #numero de columnas
m = A.shape[0] #numero de filas
nmax =n*(n-1)/2

In [39]:
#inicializa valores del ciclo
ak = A
vk = np.identity(n, dtype = float) 
sig = ''
uk = ak
num_col_ortogonal = 0
k = 0
stop = False


### Borrador

In [40]:
ak = a
col_j = ak[:,1]
col_i = ak[:,0]
col_j

array([ 0.42890649,  0.13255354, -0.20105594,  0.17667344, -0.42833774,
        0.55114437, -0.41428721, -0.07969703, -0.24821956])

In [41]:
x1 = [1,2,3,4,5,6,7,8,9]

In [42]:
np.linalg.norm(x1)

16.881943016134134

In [43]:
np.sqrt(np.dot(x1,x1))

16.881943016134134

In [44]:
(np.linalg.norm(col_j,2))

1.0

In [45]:
np.sqrt(np.dot(col_j,col_j))

1.0

In [46]:
len(col_j)

9